#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: spaceIndex

Variabili utili per testare il funzionamento


In [17]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [18]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## FUNZIONI AGGIUNTIVE

### CreateIntervalTree

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [19]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [20]:
t = createIntervalTree(dict)  # Creazione dell'albero

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

### RemoveIntersection

rimuove le intersezioni contenute in **covers** che i boundingbox hanno con se stessi


In [21]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

## addIntersection

Funzione che aggiunge gli elementi di iterator nell'i-esimo array di covers. Utilizzata in: spaceindex, boxcovering

In [22]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

## Funzioni di supporto

## boundingbox

La funzione boundingbox serve a creare il bounding Box di una cella, cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [23]:
function boundingboxMOD(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
    if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingboxMOD (generic function with 1 method)

### boxcovering

boxcovering calcola quali bounding box si intersecano tra loro.

In [24]:
function boxcoveringMOD(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcoveringMOD (generic function with 1 method)

### coordIntervals


coordintervals crea un dizionario ordinato dove la chiave è l'intervallo su una coordinata, e come valore associato ha l'indice dell'intervallo corrispondente nel boundig box

In [25]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

## Versione iniziale di spaceindex

spaceindex, dato un modello, restituisce un array di array dove l'elemento i-esimo rappresenta quali intersezioni ha il bounding box i-esimo con gli altri bounding box

In [26]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end


spaceindex (generic function with 1 method)

In [27]:
@btime spaceindex((V,EV))       #108,350 μs

  106.521 μs (1102 allocations: 49.08 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

Attraverso la macro  **@code_warntype** abbiamo scoperto che spaceIndex è type-unstable

### Versione parallelizzata di SpaceIndex

In [28]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees

function spaceindexMOD(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    dim = size(V,1)
    
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]		    #calcola le celle
    bboxes = [hcat(boundingboxMOD(cell)...) for cell in cellpoints]    #calcola i boundingbox delle celle
    
    xboxdict = Threads.@spawn coordintervals(1,bboxes)
    yboxdict = Threads.@spawn coordintervals(2,bboxes)

    # xs,ys sono di tipo IntervalTree
    xs = Threads.@spawn createIntervalTree(fetch(xboxdict))
    ys = Threads.@spawn createIntervalTree(fetch(yboxdict))
    
    xcovers = Threads.@spawn boxcoveringMOD(bboxes, 1, fetch(xs))                        #lista delle intersezioni dei bb sulla coordinata x
    ycovers = Threads.@spawn boxcoveringMOD(bboxes, 2, fetch(ys))                        #lista delle intersezioni dei bb sulla coordinata x
    covers = [intersect(pair...) for pair in zip(fetch(xcovers),fetch(ycovers))]      #lista delle intersezioni dei bb su entrambe le coordinate

    if dim == 3
        zboxdict = Threads.@spawn coordintervals(3,bboxes)
        zs = Threads.@spawn createIntervalTree(fetch(zboxdict))
        zcovers = Threads.@spawn boxcoveringMOD(bboxes, 3, fetch(zs))
        covers = [intersect(pair...) for pair in zip(fetch(zcovers),covers)]
    end
    
    removeIntersection(covers)       #rimozione delle intersezioni con se stesso
    return covers
end

spaceindexMOD (generic function with 1 method)

In [29]:
@btime spaceindexMOD((V, EV))       #108,182 μs

  162.880 μs (538 allocations: 37.27 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

Dal code_warntype di spaceIndex emerge l'instabilità riguardo alcune variabili e non dell'intero metodo. Si procede nell'analisi delle singole funzioni mono-tasks.

## TEST

In [34]:
using Test

@testset "Refactoring spaceindex tests" begin
    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    EV = [[1, 2], [3, 4], [5, 6], [1, 3], [2, 4], [3, 5], [4, 6]]
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    xboxdict = Lar.coordintervals(1,bboxes)
    yboxdict = Lar.coordintervals(2,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = Lar.boxcovering(bboxes, 1, xs)
    ycovers = Lar.boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]
    
    @test covers == Array{Int64,1}[[1, 4, 5], [4, 5, 2, 6, 7], [6, 7, 3], 
        [1, 4, 2, 6], [1, 5, 2, 7], [4, 2, 6, 3], [5, 2, 7, 3]]
end

Test Summary:                | Pass  Total
Refactoring spaceindex tests |    1      1


Test.DefaultTestSet("Refactoring spaceindex tests", Any[], 1, false, false)

![test di spaceindex](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/spaceindex_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [35]:
@benchmark spaceindex((V, EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  163.309 μs …  34.021 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     186.264 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   293.186 μs ± 659.973 μs  ┊ GC (mean ± σ):  2.53% ± 2.50%

  █▅▃▃▂▁▁                                                       ▁
  █████████▇▇▇▇▇▇▆▆▆▆▅▆▅▆▅▅▄▄▅▅▅▅▄▅▅▃▅▄▅▄▅▃▅▅▃▃▄▃▃▅▄▅▄▃▃▄▃▄▃▂▃▃ █
  163 μs        Histogram: log(frequency) by time       2.33 ms <

 Memory estimate: 37.27 KiB, allocs estimate: 538.

funzione modificato:

In [36]:
@benchmark spaceindexMOD((V, EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  107.142 μs …  12.075 ms  ┊ GC (min … max): 0.00% … 96.96%
 Time  (median):     111.413 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   143.821 μs ± 316.102 μs  ┊ GC (mean ± σ):  6.10% ±  2.91%

  █▅▃▁           ▁                                              ▁
  ███████▇█▇▇▇▇███████▇█▇▇▇▇▇▇▇▆▆▆▅▆▅▆▆▆▅▅▅▅▅▅▅▆▅▅▆▅▄▅▅▅▄▄▄▄▅▁▄ █
  107 μs        Histogram: log(frequency) by time        562 μs <

 Memory estimate: 49.08 KiB, allocs estimate: 1102.